In [ ]:
S <- 1e5
boot <- numeric(S)

for (i in 1:S) {
    sample_data <- sample(data, size = n, replace = TRUE)
    boot[i] <- statistic(sample_data)
}

# Contigency Table

In [ ]:
AL <- c(rep("Agree", 169), rep("Disagree", 196))
Uni <- c(rep("Agree", 363), rep("Disagree", 557))

diff_boot <- replicate(10000, {
  AL_sample <- sample(AL, length(AL), replace = TRUE)
  Uni_sample <- sample(Uni, length(Uni), replace = TRUE)

  p_agree_AL <- mean(AL_sample == "Agree")
  p_agree_Uni <- mean(Uni_sample == "Disagree")

  p_agree_AL - p_agree_Uni
})

# Odds Ratio

In [ ]:
bootstrap_table <- function(tab, statistic, R = 1000, conf = 0.95) {
  N <- sum(tab)
  p_hat <- as.vector(tab) / N
  boot_stats <- numeric(R)

  for (i in 1:R) {
    new_counts <- rmultinom(1, size = N, prob = p_hat)
    new_tab <- matrix(new_counts, nrow = nrow(tab), ncol = ncol(tab))
    boot_stats[i] <- statistic(new_tab)
  }

  stat_hat <- statistic(tab)

  alpha <- (1 - conf) / 2
  tci <- mean(boot_stats) + qt(c(alpha, 1 - alpha), df = R - 1) * sd(boot_stats) / sqrt(R)
  pci <- quantile(boot_stats, probs = c(alpha, 1 - alpha))

  return(list(
    estimate = stat_hat,
    boot_mean = mean(boot_stats),
    boot_se = sd(boot_stats),
    t_conf_interval = tci,
    p_conf_interval = pci,
    boot_samples = boot_stats
  ))
}

tab <- matrix(c(
  12, 8,
  5, 15
), nrow = 2, byrow = TRUE)


odds_ratio <- function(tab) {
  (tab[1, 1] * tab[2, 2]) / (tab[1, 2] * tab[2, 1])
}


res <- bootstrap_table(tab, odds_ratio, R = 2000)
res

hist(res$boot_samples,
  probability = TRUE, col = "lightblue", breaks = 30,
  main = "Bootstrap Distribution of Odds Ratio"
)
abline(v = res$estimate, col = "red", lwd = 2)